In [ ]:
import os
import gmsh
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from fem_pre_processing import read_mesh
from fem_processing import gaussian_quadrature, matrices_assembly as assembly
from fem_pos_processing import graph_results as graph

# `get_edge_key_map()`

In [ ]:
def get_edge_key_map(mesh_data):
    # Construir um dicionário auxiliar para mapear arestas diretamente às suas chaves
    edge_key_map = {}
    for key, edge in mesh_data['edges'].items():
        # Usamos uma tupla ordenada para representar a aresta, garantindo unicidade independente da direção
        edge_nodes = tuple(sorted(edge['conn']))
        edge_key_map[edge_nodes] = key
    return edge_key_map

# `apply_physics()`

In [2]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']
    edges_data = mesh_data['edges']
    
    # Mapeamento das arestas diretamente às suas chaves    
    edge_key_map = get_edge_key_map(mesh_data)

    for key, cell in mesh_data['cell'].items():
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []

        # Geometric data    
        xc = np.mean([mesh_data['nodes'][node]['xg'][0] for node in cell['conn']])
        yc = np.mean([mesh_data['nodes'][node]['xg'][1] for node in cell['conn']])
        Je = assembly.vectorial_jacobian(mesh_data, cell)
        area = np.abs(np.linalg.det(Je)) * 0.5 
        cell['geo']['centroid'] = (xc, yc)
        cell['geo']['area'] = area

        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Adicionar a nova chave 'material' ao dicionário da célula
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
        for xik in gauss_points: 
            xg_e, yg_e = assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(np.eye(2))
            cell['mass_term'].append(1)           
            cell['source'].append(np.zeros((2, 1)))

        # Contorno da fronteira
        gamma_a = {idx: node for idx, node in enumerate(cell['conn'])
                        if nodes_data[node]['bc']['tag'] == 101}  
        if len(gamma_a) > 1:
            cell_data[key]['contour'] = {'type': 'Dirichlet', 'conn_contour': gamma_a}

        # Atualiza a conectividade de arestas da célula
        cell['conn_edge'] = [
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][1]]))],  # e1: nó 1 -> nó 2
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][2]]))],  # e2: nó 1 -> nó 3
            edge_key_map[tuple(sorted([cell['conn_sorted'][1], cell['conn_sorted'][2]]))]   # e3: nó 2 -> nó 3
        ]

        # Atualiza dados físicos das arestas
        for key, edge in edges_data.items():
            # Nós da aresta
            n0, n1 = edge['conn'][0], edge['conn'][1]
            
            # Adiciona os potenciais de Dirichlet sobre as arestas
            if nodes_data[n0]['bc']['type'] == 'Dirichlet' and nodes_data[n1]['bc']['type'] == 'Dirichlet':
                edge['bc'] = {
                    'tag': 101,
                    'type': 'Dirichlet',
                    'value': nodes_data[n0]['bc']['value'],
                    'name': 'contour_domain'}
                
            # Coordenadas dos nós inicial e final
            xe_0, ye_0 = mesh_data['nodes'][n0]['xg']
            xe_1, ye_1 = mesh_data['nodes'][n1]['xg']
        
            # Adiciona o tamanho da aresta
            edge['len'] = ((xe_1 - xe_0)**2 + (ye_1 - ye_0)**2) ** 0.5
    
    return mesh_data

# `create_rectangular_guide()`

In [ ]:
def create_rectangular_guide(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT

    # Dimensões do guia de onda retangular
    a, b = 8e-2, 4e-2 

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_guide")

    # Criar superfície retangular
    TagSurface = gmsh.model.occ.addRectangle(0, 0, 0, a, b)
    gmsh.model.occ.synchronize()
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.model.mesh.generate(dim=2)
    gmsh.model.mesh.setOrder(order)

    # Obter os contornos (curvas, dim=1) de cada superfície
    outDimTags = gmsh.model.getBoundary([(2, TagSurface)], oriented=True, recursive=False)

    # Exibir os TAGs das curvas associadas a cada contorno
    tagList_boundary = [Dimtags[1] for Dimtags in outDimTags]

    # Definindo as curvas de contorno de Dirichlet (dim=1)
    gmsh.model.addPhysicalGroup(dim=1, tags=tagList_boundary, tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

    # Adicionar grupos físicos para Dim=2 (superfícies)
    gmsh.model.addPhysicalGroup(dim=2, tags=[TagSurface], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_guide_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    # Finalize the gmsh model e return the mesh data
    gmsh.finalize()
    return mesh_data

# `waveguide_modes()`

In [ ]:
def waveguide_modes(mn_max, mode_type):
    """
    Calcula os modos TE ou TM para um guia de onda retangular.

    Parâmetros:
    a (float): Dimensão transversal a (em metros).
    b (float): Dimensão transversal b (em metros).
    m_max (int): Número máximo de modos para m.
    n_max (int): Número máximo de modos para n.
    mode_type (str): Tipo de modo, 'TE' ou 'TM'.

    Retorna:
    dict: Dicionário com os modos e seus respectivos kc.
    """
    # Dimensões do guia de onda retangular
    a, b = 8e-2, 4e-2 

    if mode_type not in ['TE', 'TM']:
        raise ValueError("mode_type deve ser 'TE' ou 'TM'")

    modes = {}
    for m in range(mn_max + 1):
        for n in range(mn_max + 1):
            if mode_type == 'TE' and m == 0 and n == 0:
                continue  # Excluir o modo (0,0) para TE
            if mode_type == 'TM' and (m == 0 or n == 0):
                continue  # Excluir modos onde m=0 ou n=0 para TM
            kc = np.sqrt((m * np.pi / a)**2 + (n * np.pi / b)**2)
            modes[f"{mode_type}_{m}{n}"] = kc**2

    # Ordenar os autovalores e listar os cinco menores
    sorted_te_modes = sorted(modes.items(), key=lambda x: x[1])
    smallest_modes = {key: value 
                      for i, (key, value) in enumerate(sorted_te_modes) if i < mn_max}

    # Imprimir os menores autovalores
    print(f"\n{mn_max} menores autovalores analíticos dos modos {mode_type}:")
    for mode, kc2 in smallest_modes.items():
        print(f"{mode}: kc2 = {kc2:.3f} rad2/m2")
    
    return smallest_modes

# `calculate_error()`

In [2]:
def calculate_error(modes_u, modes_uh):
    """
    Calcula o erro entre os valores de referência (modos analíticos) e os modos computados numericamente.

    Parâmetros:
    reference_modes (list): Lista de tuplas representando os modos analíticos, onde cada tupla contém o nome do modo e o valor de kc.
    computed_modes (list): Lista de valores computados numericamente para os menores kc^2.

    Retorna:
    dict: Um dicionário contendo os erros absolutos para cada modo correspondente.
    """
    # Compatibilizar tamanhos
    smallest_modes_uh = modes_uh[:len(modes_u)]
    
    # Calcular os erros percentuais
    errors = [abs(ref - comp)/ref * 100
               for ref, comp in zip(modes_u.values(), smallest_modes_uh)]

    # Retornar os erros em um dicionário
    errors_dict = {mode: error for mode, error in zip(modes_u.keys(), errors)}

    # Imprimir dicionário de erros
    for mode, error in errors_dict.items():
        print(f"Modo {mode}: Erro percentual = {error:.4f}")

    return errors_dict

# `potential_and_gradient()`

In [4]:
def potential_and_gradient(x, y):
    return 0, np.zeros((2,1))

# `get_modes_pattern()`

In [ ]:
def get_modes_pattern(mesh_data, eigenvalues, eigenvectors, modes, type='abs'):
    # Solução aproximada
    for i, eigenvalue in enumerate(eigenvalues):
        # Solução aproximada global
        uh = matrices_assembly.global_potentials_solution(mesh_data, eigenvectors[:, i])

        # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
        xg, yg, conn = graph.structured_data(mesh_data)

        # Para elementos P1 (3 nós por elemento)
        triangulation = Triangulation(xg, yg, conn)

        # Conversão do dicionário de soluções para lista
        if type == 'real':
            uh = np.real(list(uh.values()))
        elif type == 'imag':
            uh = np.imag(list(uh.values()))
        elif type == 'abs':
            uh = np.abs(list(uh.values()))
        else:
            raise ValueError("Tipo de solução inválida. Use 'real', 'imag' ou 'abs'.")
    
        plt.figure(figsize=(8, 6))
        plt.tricontour(triangulation, uh, cmap='viridis')
        plt.title(rf'Solução Aproximada para Autovalor $\lambda_{i}$ = {eigenvalue:.3f}. Modo {modes[i][0]}')
        plt.xlabel(r'$x$')
        plt.ylabel(r'$y$')
        plt.tight_layout()

# `plot_dispersive_curve()`

In [ ]:
def plot_dispersive_curve(eigenvalues):

    # Constante de onda de corte do modo TE10 (1/m)
    kc_10 = np.sqrt(eigenvalues[0])   

    # Varredura de k (# Evita k=0)
    k = np.linspace(1 * kc_10, 4 * kc_10, 500)  

    # Cálculo da dispersão para o modo TE10
    beta = np.sqrt(k**2 - kc_10**2)  

    # Normalização
    k_normalized = k / kc_10
    beta_normalized = beta / k

    # Plota o gráfico de dispersão
    plt.figure(figsize=(8, 6))
    plt.plot(k_normalized, beta_normalized, label="Modo TE10", color="blue")

    # Destaque da faixa de operação monomodo
    plt.axvline(1, color="red", linestyle="--", label="k = k_c10")
    plt.axvline(4, color="green", linestyle="--", label='k = 4k_c10')

    # Configurações do gráfico
    plt.xlabel(r"$k/k_{c10}$", fontsize=12)
    plt.ylabel(r"$\beta/k$", fontsize=12)
    plt.title("Curva de Dispersão do Modo TE10", fontsize=14)
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()  

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.